In [22]:
import random
import copy
import collections
from pgmpy.sampling import BayesianModelSampling
class Thing:
    def __init__(self,name,level):
        self.name=name
        self.level=level

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))
    
    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive
   
    def get_level(self):
        return self.level
    def set_level(self, l):
        self.level=l
        
class Food(Thing):
    location=1
    pass            
        
class Symptoms(Thing):
    location=1
    pass

 

In [23]:
class FoodAndSymptoms(): 
    """Environment"""
 
    def __init__(self, foods,symp):
        self.things= foods+symp
        self.foods = foods
        self.symptoms = symp
        self.agents = []
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        symptoms=[]
        foods=[]
        for p in things: 
            if isinstance(p, Symptoms):
                symptoms.append(p)
            if isinstance(p, Food):
                foods.append(p)
        return symptoms+foods
    
    def execute_action(self, agent, action):
        for a in action:
            (act,food,n)=a
            if act == "increase":
                print('{} decided to {} '.format(str(agent)[1:-1], action))
                agent.increase(food,n)
            elif act == "decrease":
                print('{} decided to {} '.format(str(agent)[1:-1], action))
                agent.decrease(food,n)
            
    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def add_agent(self, agent):
        self.agents.append(agent)
        
    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        return [thing for thing in self.things
                if thing.location == location and isinstance(thing, tclass)]
        
    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()
        return actions
    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

  
                    
             
    

In [24]:
class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(
                self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

In [25]:
class Person(Agent):
    location =1
    """Aumenta el consumo"""
    def increase(self, thing,n):
        if isinstance(thing, Food):
            thing.set_level(thing.get_level+n)
    """disminuye consumo"""    
    def decrease(self, thing,n):
        if isinstance(thing, Food):
            thing.set_level(thing.get_level-n)


In [26]:
import random
random.seed(1)
def program(percepts):
    actions=[]
    for p in percepts: 
        if isinstance(p, Food) :
            if random.randint(0,1)==1:
                if p.get_level()==2:
                    actions.append( ("decrease", p.name,random.randint(0, 2)))
                elif p.get_level()==1:
                    if random.randint(0,1)==0:
                        actions.append(("decrease", p.name,1))
                    else:
                        actions.append(("increase",p.name,1))
                else:
                    actions.append(("increase",p.name,random.randint(0, 2)))
    return actions

    

In [27]:
import pandas as pd
alimentos= pd.read_csv("alimentos.csv", delimiter=';')
allfoods=[]
for a in alimentos['alimento']:
    f=Food(a,0)
    allfoods.append(f)

In [28]:
sintomas=pd.read_csv("sintomas.csv", delimiter=';')
allsymptoms=[]
for b in sintomas['symptom']:
    s=Symptoms(b,0)
    allsymptoms.append(f)

In [29]:
fodmapsEnvironment=FoodAndSymptoms(allfoods,allsymptoms)

In [30]:
pacient=Person(program)
fodmapsEnvironment.add_agent(pacient)

In [31]:
diet=pd.read_csv("personDiet.csv")

for a in list(diet):
    for b in sintomas['symptom']:
        if(a==b):
            thing=Symptoms(a,diet.iloc[0][a])
        else:
            thing=Food(a,diet.iloc[0][a])
    fodmapsEnvironment.add_thing(thing,1)
diet

,Unnamed: 0,pallar,vainitas,pepinillo,zanahoria,zapallo,yogurt,vino,venado,uva,...,aji,sintomasDispepsicos,frijol,flatulencia,cafe,dolorAbdominal,reflujoGastroesofagico,cebolla,borborigmos,distensionAbdominal
0,0,0,0,0,0,0,2,0,0,0,...,1,0,1,1,2,2,0,1,0,1


In [20]:
decision=fodmapsEnvironment.step()

Person decided to [('increase', 'albahaca   ', 0), ('increase', 'apio       ', 1), ('increase', 'brocoli    ', 2), ('increase', 'caigua     ', 0), ('increase', 'camucamu   ', 0), ('increase', 'coco       ', 1), ('increase', 'granadilla ', 1), ('increase', 'limon      ', 0), ('increase', 'maracuya   ', 2), ('increase', 'naranja    ', 2), ('increase', 'arroz      ', 1), ('increase', 'cebolla    ', 0), ('increase', 'chirimoya  ', 1), ('increase', 'conejo     ', 2), ('increase', 'huevo      ', 0), ('increase', 'jamon      ', 2), ('increase', 'leche_vaca ', 2), ('increase', 'lechuga    ', 1), ('increase', 'lentejas   ', 2), ('increase', 'mantequilla', 2), ('increase', 'palta      ', 0), ('increase', 'pavo       ', 1), ('increase', 'papa       ', 2), ('increase', 'pejerrey   ', 0), ('increase', 'pera       ', 2), ('increase', 'pimiento   ', 1), ('increase', 'pollo      ', 2), ('increase', 'quinua     ', 0), ('increase', 'salchicha  ', 2), ('increase', 'te         ', 2), ('increase', 'venado 

In [21]:
decision

[[('increase', 'albahaca   ', 0),
  ('increase', 'apio       ', 1),
  ('increase', 'brocoli    ', 2),
  ('increase', 'caigua     ', 0),
  ('increase', 'camucamu   ', 0),
  ('increase', 'coco       ', 1),
  ('increase', 'granadilla ', 1),
  ('increase', 'limon      ', 0),
  ('increase', 'maracuya   ', 2),
  ('increase', 'naranja    ', 2),
  ('increase', 'arroz      ', 1),
  ('increase', 'cebolla    ', 0),
  ('increase', 'chirimoya  ', 1),
  ('increase', 'conejo     ', 2),
  ('increase', 'huevo      ', 0),
  ('increase', 'jamon      ', 2),
  ('increase', 'leche_vaca ', 2),
  ('increase', 'lechuga    ', 1),
  ('increase', 'lentejas   ', 2),
  ('increase', 'mantequilla', 2),
  ('increase', 'palta      ', 0),
  ('increase', 'pavo       ', 1),
  ('increase', 'papa       ', 2),
  ('increase', 'pejerrey   ', 0),
  ('increase', 'pera       ', 2),
  ('increase', 'pimiento   ', 1),
  ('increase', 'pollo      ', 2),
  ('increase', 'quinua     ', 0),
  ('increase', 'salchicha  ', 2),
  ('increase',

In [21]:
evidence=[]
for i in decision:
    for j in i:
        (a,b,c)=j
        evidence.append((b,c))

In [22]:
evidence

[('albahaca   ', 0),
 ('apio       ', 1),
 ('brocoli    ', 2),
 ('caigua     ', 0),
 ('camucamu   ', 0),
 ('coco       ', 1),
 ('granadilla ', 1),
 ('limon      ', 0),
 ('maracuya   ', 2),
 ('naranja    ', 2),
 ('arroz      ', 1),
 ('cebolla    ', 0),
 ('chirimoya  ', 1),
 ('conejo     ', 2),
 ('huevo      ', 0),
 ('jamon      ', 2),
 ('leche_vaca ', 2),
 ('lechuga    ', 1),
 ('lentejas   ', 2),
 ('mantequilla', 2),
 ('palta      ', 0),
 ('pavo       ', 1),
 ('papa       ', 2),
 ('pejerrey   ', 0),
 ('pera       ', 2),
 ('pimiento   ', 1),
 ('pollo      ', 2),
 ('quinua     ', 0),
 ('salchicha  ', 2),
 ('te         ', 2),
 ('venado     ', 2),
 ('vino       ', 2),
 ('yogurt     ', 1),
 ('zapallo    ', 2),
 ('pepinillo  ', 2),
 ('pallar   ', 0),
 ('pallar   ', 1),
 ('pallar   ', 1),
 ('pallar   ', 1),
 ('pallar   ', 0),
 ('pallar   ', 1),
 ('vainitas', 0),
 ('yogurt', 0),
 ('vino', 0),
 ('venado', 0),
 ('sardina', 1),
 ('salchicha', 1),
 ('quinua', 2),
 ('queso', 1),
 ('pollo', 1),
 ('pi